In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from pandas.tseries.offsets import BDay
from rapidfuzz import process


def get_file_path(relative_path):
    parent_dir = os.getcwd()
    return os.path.join(parent_dir, relative_path)

In [2]:
def parse_dates(date_str):

    for fmt in ('%d.%m.%Y', '%d/%m/%Y', '%Y-%m-%d'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        
        except ValueError:
            continue

    return pd.NaT

actual_index = 'Company Name'
approximate_index = 'Employer'
start_date_index = 'Start Date'
isin_index = 'ISIN Code'
ipo_date_index = 'IPO Date'

original_path = get_file_path('data/all_us.csv')
test_path = get_file_path('GLOBAL_TEST.csv')
dirty_data = get_file_path('work_stoppages.csv')

actual = pd.read_csv(original_path, dtype=str)
actual_list = actual[actual_index].tolist()

# Dirty dataset
approximate = pd.read_csv(dirty_data, dtype=str)
approximate['Start Date'] = pd.to_datetime(approximate['Start Date'], format='%Y-%m-%d')
approximate['End Date'] = pd.to_datetime(approximate['End Date'], format='%Y-%m-%d')
approximate['Request Start Date'] = approximate['Start Date'] - pd.DateOffset(years=1)
approximate['Request End Date'] = approximate['End Date'] + pd.DateOffset(years=1)
approximate['Duration'] = approximate['End Date'] - approximate['Start Date']

# Attempt to convert date strings to datetime 
actual['IPO Date'] = actual['IPO Date'].apply(parse_dates)

# Handle exceptions, use incorporation dates as proxy for ipo dates
nat_mask = actual['IPO Date'].isna()
nat_rows = actual[nat_mask].copy()
nat_rows['IPO Date'] = nat_rows['Date of Incorporation'].apply(parse_dates)
actual.loc[nat_mask, 'IPO Date'] = nat_rows['IPO Date']

approximate['isin'] = np.nan

rows_to_drop = []
new_rows = []

# Matching company names to ISIN codes with fuzzywuzzy
for index, row in approximate.iterrows():

    company = row[approximate_index]
    match = None
    
    try:
        match = process.extractOne(company, actual_list, score_cutoff=90)

    except:
        pass

    if match:

        company_name = match[0]
        extraction = actual.loc[actual[actual_index] == company_name, (isin_index, ipo_date_index)].iloc[0]
    
        ipo_date = extraction.iloc[1]
        strike_date = row[start_date_index]
        
        if strike_date > ipo_date:

            isin_number = extraction.iloc[0]
            rsd = row['Request Start Date']
            red = row['Request End Date']
            ni = row['# Idled']
            dur = row['Duration'].days
            ind = row['Industry']

            actual.loc[index, approximate_index] = company_name
            actual.loc[index, 'isin'] = isin_number

            
            # print(company_name, isin_number, strike_date.strftime('%m/%d/%Y'), index+2, match[1])
            new_row = {'Company Name': company_name, 'ISIN': isin_number, 'Industry': ind, '# Idled': ni, 'Start Date': strike_date.strftime('%m/%d/%Y'), 'Duration': dur, 'Request Start Date': rsd, 'Request End Date': red, 'Original Index': index+2}
            new_rows.append(new_row)


        else:
            rows_to_drop.append(index)

    else:
        rows_to_drop.append(index)

# print(rows_to_drop)
approximate.drop(rows_to_drop, inplace=True)

new_data = pd.DataFrame(new_rows)

new_data.to_csv('data/work_stoppages_showcase.csv')


The data was then used to pas refinitif eikon request in CEDIF. A column specifying the request reception date was added.

Various data were obtained and placed into .csv files to be used in the section that follows. Here we combine and process the data such that it can be turned into a panel dataset. We opt to save the intermittent results to .csv files for easy troubleshooting.

In [2]:
# Tool functions
def adjust_to_business_day(date, data_index):

    original_date = date 
    max_attempts = 10 
    attempts = 0

    while date not in data_index and attempts < max_attempts:
        date += BDay()
        attempts += 1

    if attempts == max_attempts:

        print(f"Couldn't find a suitable business day for original date: {original_date}")
        raise ValueError("Couldn't find a suitable business day in the index after 10 attempts.")
    
    return date

def open_csv(filepath):
    try:
        return pd.read_csv(filepath)
    except FileNotFoundError:
        return None

def load_and_prepare_data(filepath, skiprows=1, date_col='Date', date_format='%Y', sort_index=True):
    df = pd.read_csv(filepath, skiprows=skiprows)
    df.rename(columns={df.columns[0]: date_col}, inplace=True)
    df[date_col] = pd.to_datetime(df[date_col], format=date_format)
    df.set_index(date_col, inplace=True)
    # if sort_index:
    df.sort_index(inplace=True)
    return df[~df.index.isna()]

def get_data_for_isin_and_date(df, isin, date):
    data = None
    closest_year = df.index.asof(date)
    try:
        data = df.loc[closest_year, isin]
    except KeyError:
        pass
    return data

def prepare_data(df):
    df = df.drop(df.columns[:2], axis=1).T.reset_index()
    df.columns = ['Date', 'Price']
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    return df.dropna(subset=['Date']).set_index('Date')

def process_and_save_df(df, file_path, mask_all_nan):
    df_processed = df[~mask_all_nan].fillna(0).reset_index()
    df_processed = df_processed.rename(columns={'index': 'Date'})
    file_path = get_file_path(file_path)
    df_processed.to_csv(file_path, index=False)

# Logic functions
def changes(df, start_date, end_date, isin) -> dict:

    df_index = df.index

    # In this section more factors can be added
    workers = get_data_for_isin_and_date(emp, isin, start_date)
    # print(workers)
    if workers is None or pd.isna(workers):
        workers = 0

    capital = get_data_for_isin_and_date(cap, isin, start_date)
    if capital is None:
        capital = 0

    # Start/End dates with a month offset
    start_date = adjust_to_business_day(start_date - pd.DateOffset(months=1), df_index)
    end_date = adjust_to_business_day(end_date + pd.DateOffset(months=1), df_index)

    # Fetch the stock prices for the period, calulate returns
    price_slice = df.loc[start_date:end_date, 'Price'].pct_change()
    price_slice.replace(0, np.nan, inplace=True)
    price_slice.fillna(method='ffill', inplace=True)

    # Create a series as long as the price_slice period
    workers_slice = pd.Series(workers, index=price_slice.index)
    capital_slice = pd.Series(capital, index=price_slice.index)
    # print(workers_slice)

    isin_str = str(isin)

    # The output dictionary, add factors here with key-value pairs, keys are not important
    # Adjust the for loop to handle additional data right after the get_y() function is invoced
    return {isin_str: price_slice, 'workers': workers_slice, 'capital': capital_slice}

def get_y(isin, index, start_date, end_date):
    filepath = f'all_csv/{isin.upper()}_{index}.csv'
    df = open_csv(get_file_path(filepath))
    if df is None:
        return None
    df_prices = prepare_data(df)
    return changes(df_prices, start_date, end_date, isin)

def get_x(idle, duration, industry):
    # DEPRECATED
    return [idle, duration, industry]

# Main sequence
# Loading data
emp_path = 'data/ds_employees.csv'
cap_path = 'data/ds_cintensity.csv'
filepath = 'data/work_stoppages_extracted.csv'
sp_path = 'data/S&P_ds.csv'

emp = load_and_prepare_data(get_file_path(emp_path))
cap = load_and_prepare_data(get_file_path(cap_path))

data = pd.read_csv(get_file_path(filepath))
data['Request Start Date'] = pd.to_datetime(data['Request Start Date'], errors='coerce')
data['Request Recieved Date'] = pd.to_datetime(data['Request Start Date'], errors='coerce')
data['Start Date'] = data['Request Start Date'] + pd.DateOffset(years=1)

# Master date range index to allign the unbalanced pooled dataset
master_date_range = pd.date_range(start='1982-1-1', end='2021-12-31', freq='D')
all_slices_combined = pd.DataFrame(index=master_date_range)
strike_dummy_slices = pd.DataFrame(index=master_date_range)
relative_idle_slices = pd.DataFrame(index=master_date_range)
c_intesity_slices = pd.DataFrame(index=master_date_range)

leadup_slices = pd.DataFrame(index=master_date_range)
followup_slices = pd.DataFrame(index=master_date_range)

# S&P returns, special treatment round 1
sp = pd.read_csv(get_file_path(sp_path), usecols=[0, 1], parse_dates=[0], dayfirst=True)
sp.set_index('Date', inplace=True)
sp = sp.pct_change()
sp = sp.reindex(master_date_range)
sp.replace(0, np.nan, inplace=True)
sp.fillna(method='bfill', inplace=True)

# Result list
time_slices_info = []

n = 0
for index, row in data.iterrows():
    
    # For limitng recursion depth
    # if n == 30:
    #         break
    # n += 1

    # Can not continue if duration value is missing
    try:
        duration = int(row['Duration'])
    except:
        duration = None
        continue
    
    # Fetching data for each entry
    isin = row['ISIN']
    idle = row['# Idled']
    industry = row['Industry']
    start_date = row['Start Date']
    end_date = start_date + pd.DateOffset(days=duration)
    request_date = row['Request Recieved Date']

    # Each tuple in this list has all the data needed to call the get_y() function
    # The list contains the tuples corresponding to each entry
    time_slices_info.append((start_date, end_date, isin, index+1, int(idle)))


for i in time_slices_info:
    start_date, end_date, isin, index, idle = i
    # Load the data for the time slice
    slice_dict = get_y(isin, index, start_date, end_date)

    # Dependent variable section
    if slice_dict is None:
            continue
    (isin, slice_data), (z, total_employees), (zz, c_intesity)  = slice_dict.items()
    relative_idle = idle / total_employees
    relative_idle[np.isinf(relative_idle)] = 0
    # print(relative_idle)

    # Reindex the slice_data to the master date range
    slice_reindexed = slice_data.reindex(master_date_range)
    relative_idle = relative_idle.reindex(master_date_range)
    c_intesity = c_intesity.reindex(master_date_range)
    # print(relative_idle.isna().all())
    
    if isin in all_slices_combined.columns:
        all_slices_combined[isin].update(slice_reindexed)
    else:
        all_slices_combined[isin] = slice_reindexed

    # Dummy variable section, take in strike start-end dates
    current_slice_range = pd.date_range(start=start_date, end=end_date)
    # Set all values in that range to 1
    dummy_series = pd.Series(1, index=current_slice_range)
    # Align with master date index
    dummy_series = dummy_series.reindex(master_date_range)

    # Leadup dummy
    leadup = start_date - pd.DateOffset(months=1, days=1)
  
    current_slice_range = pd.date_range(start=leadup, end=start_date)
    leadup_dummy_series = pd.Series(1, index=current_slice_range)
    leadup_dummy_series = leadup_dummy_series.reindex(master_date_range)
    # leadup_dummy_series = leadup_dummy_series - dummy_series

    # Followup dummy
    followup_date = end_date + pd.DateOffset(months=1, days=1)
    current_slice_range = pd.date_range(start=end_date, end=followup_date)
    followup_dummy_series = pd.Series(1, index=current_slice_range)
    followup_dummy_series = followup_dummy_series.reindex(master_date_range)

    overlap_mask = (dummy_series == 1) & (leadup_dummy_series == 1)
    leadup_dummy_series[overlap_mask] = 0

    overlap_mask = (dummy_series == 1) & (followup_dummy_series == 1)
    followup_dummy_series[overlap_mask] = 0

    # If ISIN is not yet in the output dataframe, add ISIN and add the data
    # If ISIN is already in the output dataframe, update the (missing) values
    if isin in strike_dummy_slices.columns:
        strike_dummy_slices[isin].update(dummy_series)
    else:
        strike_dummy_slices[isin] = dummy_series

    if isin in relative_idle_slices.columns:
        relative_idle_slices[isin].update(relative_idle)
    else:
        relative_idle_slices[isin] = relative_idle

    if isin in c_intesity_slices.columns:
        c_intesity_slices[isin].update(c_intesity)
    else:
        c_intesity_slices[isin] = c_intesity

    if isin in leadup_slices.columns:
        leadup_slices[isin].update(leadup_dummy_series)
    else:
        leadup_slices[isin] = leadup_dummy_series

    if isin in followup_slices.columns:
        followup_slices[isin].update(followup_dummy_series)
    else:
        followup_slices[isin] = followup_dummy_series
    # print(relative_idle_slices.isna().all())

    # Sections can be easily replicated
    progress_value = index / len(data) * 100
    print(f'\rProgress: {progress_value:.2f}%  ', end='')

# Create a filter mask that removes any entries with missing returns data
mask_all_nan = all_slices_combined.isna().all(axis=1)

# Save the data into separate files, apply the filter mask
# process_and_save_df(all_slices_combined, 'dsf/data_for_analysis_panel.csv', mask_all_nan)
process_and_save_df(strike_dummy_slices, 'data/data_for_analysis_panel_dummy.csv', mask_all_nan)
process_and_save_df(relative_idle_slices, 'data/data_for_analysis_panel_relative_idle.csv', mask_all_nan)
process_and_save_df(c_intesity_slices, 'data/data_for_analysis_panel_cintensity.csv', mask_all_nan)

process_and_save_df(leadup_slices, 'data/data_for_analysis_panel_leadup_dummy.csv', mask_all_nan)
process_and_save_df(followup_slices, 'data/data_for_analysis_panel_followup_dummy.csv', mask_all_nan)

# S&P special treatment round 2, was not cooperating :/
sp = sp[~mask_all_nan]
# ATTENTION! Possibly wrongful forward filling of S&P returns #
sp.fillna(method='ffill', inplace=True)
###############################################################
sp = sp.fillna(0)
sp = sp.reset_index()
sp.rename(columns={'index': 'Date'}, inplace=True)
sp.rename(columns={'S&P500_CPI': 'spr'}, inplace=True)
sp.to_csv(get_file_path('data/data_for_analysis_panel_sp.csv'), index=False)

/var/folders/rp/y4zzq7f116jdmpzf6b0_rqn00000gn/T/ipykernel_28013/1462044621.py:113: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Request Start Date'] = pd.to_datetime(data['Request Start Date'], errors='coerce')


Progress: 99.13%  

Adding control variables and generating miscellaneous variables for visualization purposes

In [3]:
def process_data(read_path, save_path, calculation_func):
    df = pd.read_csv(read_path, skiprows=1)
    df = df.rename(columns={'ISIN codes':'Date'})
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.set_index('Date', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = calculation_func(df)
    df.fillna(0, inplace=True)
    df = df[~df.index.duplicated()]
    df = df[df.index.isin(master_date_range)]
    df.to_csv(save_path)

master_date_range = pd.date_range(start='1982-1-1', end='2021-12-31', freq='D')

# Calulation functions
def calculate_stock_returns(df):
    return df.pct_change(fill_method='pad')

def calculate_volatility(df):
    return df.rolling(window=5).std()

def calculate_volatility_of_volatility(df):
    vol = df.rolling(window=5).std()
    return vol.rolling(window=5).std()

def calculate_cumulative_returns(df):
    return (1 + df.pct_change(fill_method='pad')).cumprod() - 1

def calculate_rolling_average(df):
    return df.rolling(window=5).mean()

def plain_get(df):
    return df


# File paths, respective calls
file_paths = {
    'stock_returns': {
        'read_path': 'all_csv/stock_prices.csv', 
        'save_path': 'data/data_for_analysis_panel.csv', 
        'calculation': calculate_stock_returns
    },
    'volatility': {
        'read_path': 'all_csv/stock_prices.csv', 
        'save_path': 'data/data_for_analysis_panel_5r_volatility.csv', 
        'calculation': calculate_volatility
    },
    'volatility_of_volatility': {
        'read_path': 'all_csv/stock_prices.csv', 
        'save_path': 'data/data_for_analysis_panel_volatility_of_5r_volatility.csv', 
        'calculation': calculate_volatility_of_volatility
    },
    'cumulative_returns': {
        'read_path': 'all_csv/stock_prices.csv', 
        'save_path': 'data/data_for_analysis_panel_cumulative_returns.csv', 
        'calculation': calculate_cumulative_returns
    },
    'rolling_average': {
        'read_path': 'all_csv/stock_prices.csv', 
        'save_path': 'data/data_for_analysis_panel_5day_average.csv', 
        'calculation': calculate_rolling_average
    },
    'current_ratio': {
        'read_path': 'all_csv/currentratio.csv', 
        'save_path': 'data/data_for_analysis_panel_current.csv', 
        'calculation': plain_get
    },
    'debt_to_equity': {
        'read_path': 'all_csv/currentratio.csv',
        'save_path': 'data/data_for_analysis_panel_d2e.csv', 
        'calculation': plain_get
    }
}

for i in file_paths.items():
    target, parameters = i
    read_path = get_file_path(parameters['read_path'])
    save_path = get_file_path(parameters['save_path'])
    calculation = parameters['calculation']
    process_data(read_path, save_path, calculation)


/Users/ivankhalin/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)


This section is dedicated to combining our various .csv files into the long format panel dataset. Interestingly, we have found an easier way to retrieve stock prices for individual companies and filter them for the event windows which enabled us to circumvent the need to retrieve individual serial slices for each company.

In [4]:
stock_path = 'data/data_for_analysis_panel.csv'
stock_data = pd.read_csv(get_file_path(stock_path))
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.set_index('Date', inplace=True)
stock_data_transposed = stock_data.T
stock_melted = stock_data_transposed.reset_index().melt(id_vars=['index'], var_name='Date', value_name='return')
stock_melted.rename(columns={'index': 'ISIN'}, inplace=True)
stock_melted.set_index(['ISIN', 'Date'], inplace=True)

def process_and_merge_data(file_path, value_name, main_df):

    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data_transposed = data.T
    data_melted = data_transposed.reset_index().melt(id_vars=['index'], var_name='Date', value_name=value_name)
    data_melted.rename(columns={'index': 'ISIN'}, inplace=True)
    
    return main_df.merge(data_melted.set_index(['ISIN', 'Date']), left_index=True, right_index=True, how='left')

def add_time_varying_factor(factor_file_path, factor_name, panel_data, date_col='Date'):
   
    factor_data = pd.read_csv(factor_file_path)
    factor_data[date_col] = pd.to_datetime(factor_data[date_col])
    factor_data.set_index(date_col, inplace=True)
    panel_data_reset = panel_data.reset_index()

    panel_data_with_factor = pd.merge(panel_data_reset, factor_data[[factor_name]], left_on=date_col, right_index=True, how='left')
    panel_data_with_factor.set_index(['ISIN', date_col], inplace=True)

    return panel_data_with_factor

# Merging additional datasets with the main DataFrame
dummy_path = get_file_path('data/data_for_analysis_panel_dummy.csv')
ridle_path = get_file_path('data/data_for_analysis_panel_relative_idle.csv')
cap_path = get_file_path('data/data_for_analysis_panel_cintensity.csv')
sp_path = get_file_path('data/data_for_analysis_panel_sp.csv')
currentsave = get_file_path('data/data_for_analysis_panel_current.csv')
d2esave = get_file_path('data/data_for_analysis_panel_d2e.csv')
leadup_dummy = get_file_path('data/data_for_analysis_panel_leadup_dummy.csv')
followup_dummy = get_file_path('data/data_for_analysis_panel_followup_dummy.csv')
rv = get_file_path('data/data_for_analysis_panel_5r_volatility.csv')
vrv = get_file_path('data/data_for_analysis_panel_volatility_of_5r_volatility.csv')
cr = get_file_path('data/data_for_analysis_panel_cumulative_returns.csv')
ra = get_file_path('data/data_for_analysis_panel_5day_average.csv')

panel_data = process_and_merge_data(dummy_path, 'strike', stock_melted)
panel_data = process_and_merge_data(ridle_path, 'ridle', panel_data)
panel_data = process_and_merge_data(cap_path, 'cintensity', panel_data)
panel_data = process_and_merge_data(currentsave, 'cratio', panel_data)
panel_data = process_and_merge_data(d2esave, 'd2e', panel_data)
panel_data = process_and_merge_data(leadup_dummy, 'leadup', panel_data)
panel_data = process_and_merge_data(followup_dummy, 'followup', panel_data)
panel_data = process_and_merge_data(rv, '5day_std', panel_data)
panel_data = process_and_merge_data(vrv, 'std_of_std', panel_data)
panel_data = process_and_merge_data(cr, 'cum_returns', panel_data)
panel_data = process_and_merge_data(ra, 'rolling_avg', panel_data)

panel_data = add_time_varying_factor(sp_path, 'spr', panel_data)

panel_data = panel_data.sort_index()
panel_data = panel_data.dropna()
panel_data.replace([np.inf, -np.inf], 0, inplace=True)

#-Naive categorical industry-#
industry_path = 'data/data_for_analysis_panel_industry.csv'
industry_data = pd.read_csv(get_file_path(industry_path))
industry_data['Industry'] = industry_data['Industry'].astype('category')
panel_data = panel_data.reset_index()
panel_data = pd.merge(panel_data, industry_data[['ISIN', 'Industry']], on='ISIN', how='left')
panel_data.set_index(['ISIN', 'Date'], inplace=True)
#---------------------------#

melt_save_path = 'data/melted_panel.csv'
panel_data.to_csv(get_file_path(melt_save_path))

# Results

In [5]:
pd.set_option('display.float_format', '{:.4f}'.format)

melt_path = 'data/melted_panel.csv'
panel_data = pd.read_csv(get_file_path(melt_path), parse_dates=['Date'], index_col=['ISIN', 'Date'])

### Simplistic model

We decided to include this simplistic model as a demonstration of why it was necessary to pursue a more in depth analysis. The original code for the creation of this data file is not mentioned in this overview because, whilst it is still availible upon request, it evolved into the modules that are now showcased, and is therefore not easily includable in this overview.

In [6]:
df = pd.read_csv(get_file_path('data/data_for_analysis_naive.csv'))

print('\nEffects on pre-strike returns:')
X = sm.add_constant(df[['idle', 'duration', 'sp_pre']])  
y = df['pre']
res = sm.OLS(y, X).fit()
print(res.summary().tables[1])

print('\nEffects on returns during the strike:')
X = sm.add_constant(df[['idle', 'duration', 'sp_during']])
y = df['during']
res = sm.OLS(y, X).fit()
print(res.summary().tables[1])

print('\nEffects on post-strike returns:')
X = sm.add_constant(df[['idle', 'duration', 'sp_post']])
y = df['post']
res = sm.OLS(y, X).fit()
print(res.summary().tables[1])


Effects on pre-strike returns:
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0323      0.022      1.485      0.139      -0.011       0.075
idle         1.01e-06   3.47e-06      0.291      0.771   -5.82e-06    7.84e-06
duration      -0.0003      0.000     -1.413      0.159      -0.001       0.000
sp_pre         1.2369      0.359      3.446      0.001       0.529       1.944

Effects on returns during the strike:
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0103      0.015      0.699      0.485      -0.019       0.039
idle       -2.194e-06   2.33e-06     -0.942      0.347   -6.78e-06     2.4e-06
duration       0.0006      0.000      4.706      0.000       0.000       0.001
sp_during      0.4267      0.188      2.264      0.025      

### Naive panel regression

We turn our attention to panel regression. We already know not to include debt-to-equity and current ratio in the same specification.

In [7]:
formula = 'return - spr ~ 1 + leadup + strike + followup + ridle + cintensity + d2e'
mod = PanelOLS.from_formula(formula, panel_data)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary.tables[0])


                          PanelOLS Estimation Summary                           
Dep. Variable:                 return   R-squared:                     6.483e-06
Estimator:                   PanelOLS   R-squared (Between):              0.0218
No. Observations:              766100   R-squared (Within):            -8.61e-07
Date:                Sun, Dec 10 2023   R-squared (Overall):           6.483e-06
Time:                        18:50:03   Log-likelihood                 4.902e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.8277
Entities:                         100   P-value                           0.5481
Avg Obs:                       7661.0   Distribution:                F(6,766093)
Min Obs:                       7661.0                                           
Max Obs:                       7661.0   F-statistic (robust):             1026.5
                            

Firstly, we note that significantly imporved the imporved F-statistic results, this is a more reasonable estimation approach.

In [8]:
print(res.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0012     0.0004     3.3696     0.0008      0.0005      0.0019
leadup         0.0011     0.0013     0.8427     0.3994     -0.0014      0.0036
strike         0.0010     0.0007     1.4302     0.1527     -0.0004      0.0023
followup       0.0004     0.0011     0.3864     0.6992     -0.0017      0.0026
ridle          0.0002  1.607e-05     13.204     0.0000      0.0002      0.0002
cintensity    -0.0008     0.0006    -1.2760     0.2020     -0.0020      0.0004
d2e           -0.0001     0.0001    -1.4048     0.1601     -0.0003   5.743e-05


We remove all non-event periods to only consider the strike period +- 1 month

In [9]:
panel_data['ra'] = panel_data['return'].shift(periods=1).rolling(window=15).mean()
strike_mask = panel_data['leadup'] + panel_data['strike'] + panel_data['followup']
numeric_cols = panel_data.select_dtypes(include=[np.number])
numeric_cols = numeric_cols.multiply(strike_mask, axis=0)
non_numeric_cols = panel_data.select_dtypes(exclude=[np.number])
filtered_panel_data = pd.concat([non_numeric_cols, numeric_cols], axis=1)

### Panel with filtered data

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming 'filtered_panel_data' is your DataFrame and contains all the necessary variables
variables = filtered_panel_data[['leadup', 'strike', 'followup', 'ridle', 'cintensity', 'd2e']]

# Adding a constant term for bias
X = sm.add_constant(variables)

# Calculating VIF for each variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

     Variable    VIF
0       const 1.0197
1      leadup 1.9000
2      strike 3.0187
3    followup 1.9245
4       ridle 1.0272
5  cintensity 2.3744
6         d2e 4.0633


In [11]:
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import adfuller
# df = filtered_panel_data
# filtered_panel_data = df[(df['strike'] != 0) | (df['ridle'] != 0) | (df['cintensity'] != 0)]


# zeries = pd.read_csv(get_file_path('data_for_analysis_panel_5day_average.csv'), index_col='Date').mean(axis=0)
# adf_result = adfuller(zeries, autolag='AIC')
# print('ADF: ', adf_result[0])

formula = 'return - spr ~ leadup + strike + followup + ridle + cintensity + d2e + EntityEffects'
mod = PanelOLS.from_formula(formula, filtered_panel_data)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary.tables[0])
print(durbin_watson(res.resids))

                          PanelOLS Estimation Summary                           
Dep. Variable:                 return   R-squared:                        0.0011
Estimator:                   PanelOLS   R-squared (Between):              0.0335
No. Observations:              766100   R-squared (Within):               0.0011
Date:                Sun, Dec 10 2023   R-squared (Overall):              0.0011
Time:                        18:50:06   Log-likelihood                 2.589e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      139.58
Entities:                         100   P-value                           0.0000
Avg Obs:                       7661.0   Distribution:                F(6,765994)
Min Obs:                       7661.0                                           
Max Obs:                       7661.0   F-statistic (robust):             813.76
                            

The R squared increases significantly.

In [12]:
print(res.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
leadup         0.0030     0.0021     1.3901     0.1645     -0.0012      0.0071
strike         0.0029     0.0017     1.7016     0.0888     -0.0004      0.0063
followup       0.0025     0.0019     1.3207     0.1866     -0.0012      0.0063
ridle          0.0002   1.48e-05     16.145     0.0000      0.0002      0.0003
cintensity    -0.0006     0.0005    -1.2198     0.2225     -0.0017      0.0004
d2e           -0.0009     0.0008    -1.1257     0.2603     -0.0023      0.0006


Most p-values suggest statistical insignificance, lets adjust the specification.

In [13]:
formula = 'return - spr ~ 1 + ridle + cintensity + EntityEffects'
mod = PanelOLS.from_formula(formula, filtered_panel_data)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept   1.413e-05   2.79e-06     5.0659     0.0000   8.665e-06    1.96e-05
ridle          0.0002  4.576e-06     50.732     0.0000      0.0002      0.0002
cintensity     0.0004     0.0001     3.3742     0.0007      0.0002      0.0007


### Attempt to include categorical industry variables

In [14]:
formula = 'return - spr ~ 1 + ridle + cintensity + C(Industry)'
mod = PanelOLS.from_formula(formula, filtered_panel_data)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary.tables[1])

                                                     Parameter Estimates                                                     
                                                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                     0.0004     0.0003     1.4834     0.1380     -0.0001      0.0009
ridle                                                         0.0002  4.621e-06     50.233     0.0000      0.0002      0.0002
cintensity                                                    0.0004     0.0001     3.4470     0.0006      0.0002      0.0007
C(Industry)[T.Construction]                                  -0.0004     0.0003    -1.4649     0.1429     -0.0009      0.0001
C(Industry)[T.Health Care and Social Assistance]             -0.0004     0.0003    -1.3756     0.1689     -0.0009     

Less than ideal significance, lets try regressing each industry individually.

### Regression by industry

In [15]:
pd.set_option('display.float_format', '{:.4f}'.format)

unique_industries = filtered_panel_data['Industry'].unique()
industry_estimates = pd.DataFrame(index=unique_industries)

for industry in unique_industries:
    print(f"\n--- Regression Results for Industry: {industry} ---\n")
    industry_data = filtered_panel_data[filtered_panel_data['Industry'] == industry]

    formula = 'return - spr ~ 1 + ridle + cintensity + d2e'
    mod = PanelOLS.from_formula(formula, industry_data, check_rank=False, drop_absorbed=True)
    res = mod.fit(cov_type='clustered', cluster_entity=True)

    industry_estimates.loc[industry, 'strike'] = res.params.get('ridle', float('nan'))
    industry_estimates.loc[industry, 's_ridle'] = res.std_errors.get('ridle', float('nan'))

    print(res.summary.tables[1])


--- Regression Results for Industry: Transportation and Warehousing ---

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept   3.116e-06  1.533e-06     2.0323     0.0421   1.108e-07   6.122e-06
ridle          0.3579     0.1056     3.3903     0.0007      0.1510      0.5648
cintensity     0.0023     0.0021     1.1120     0.2662     -0.0018      0.0064
d2e           -0.0016     0.0011    -1.4507     0.1469     -0.0038      0.0006

--- Regression Results for Industry: Manufacturing ---

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept   2.921e-06  3.136e-06     0.9316     0.3515  -3.225e-06   9.067e-06


### Taking pooled regression further with interaction terms

In [16]:
formula = 'return - spr ~ 1 + leadup + strike + ridle + ra + ra:ridle + leadup:ridle + ridle:spr + strike:spr + EntityEffects'

mod = PanelOLS.from_formula(formula, filtered_panel_data)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary.tables[0])

/Users/ivankhalin/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 return   R-squared:                        0.0216
Estimator:                   PanelOLS   R-squared (Between):             -0.6042
No. Observations:              766085   R-squared (Within):               0.0216
Date:                Sun, Dec 10 2023   R-squared (Overall):              0.0216
Time:                        18:50:13   Log-likelihood                 2.597e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2118.3
Entities:                         100   P-value                           0.0000
Avg Obs:                       7660.9   Distribution:                F(8,765977)
Min Obs:                       7646.0                                           
Max Obs:                       7661.0   F-statistic (robust):             4268.8
                            

In [17]:
print(res.summary.tables[1])

                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept     8.537e-06  7.169e-06     1.1909     0.2337  -5.514e-06   2.259e-05
leadup           0.0014     0.0008     1.7514     0.0799     -0.0002      0.0030
strike           0.0009     0.0005     1.8576     0.0632  -4.845e-05      0.0018
ridle            0.0014  1.855e-05     77.323     0.0000      0.0014      0.0015
ra              -0.3164     0.0457    -6.9246     0.0000     -0.4059     -0.2268
ra:ridle        -0.0204     0.0006    -34.193     0.0000     -0.0215     -0.0192
leadup:ridle    -0.0014  2.909e-05    -47.720     0.0000     -0.0014     -0.0013
ridle:spr       -0.0395     0.0032    -12.164     0.0000     -0.0458     -0.0331
strike:spr       0.7042     0.0803     8.7692     0.0000      0.5468      0.8616
